# Simulation of Complex Systems - Chapter 12
**Author**: Artur Gasparyan

In [ ]:
need_upgrade = False
if need_upgrade:
    !pip install --upgrade matplotlib
    !pip install --upgrade networkx

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

seed = 69420
rng = np.random.default_rng(seed)

# Exercise 12.1

In [ ]:
def generate_er_graph(n_nodes, p_edge):
    er_graph = nx.DiGraph()
    er_graph.add_nodes_from(range(n_nodes))

    edges = []
    for i in range(n_nodes):
        for j in range(n_nodes):
            if rng.random() < p_edge:
                edges.append((i, j))
                #er_graph.add_edge(i, j)
                #er_graph.add_edge(j, i)
                
    er_graph.add_edges_from(edges)
    print("Generated graph with", n_nodes, p_edge)
        
    return er_graph

In [ ]:
n_nodes = 10
p_edge = 0.2

    
n_nodes_all = [20, 40, 80]
p_edges_all = [0.05, 0.04, 0.03]
er_graphs = [generate_er_graph(n_nodes, p_edge) for (n_nodes, p_edge) in zip(n_nodes_all, p_edges_all)]
n_tests = len(n_nodes_all)
fig, axes = plt.subplots(n_tests, 2, figsize=(12,12))


for i in range(n_tests):
    ax = axes[i, 0]
    nx.draw_circular(er_graphs[i], with_labels=False, ax=ax)
